# Data Wrangling & Feature Engineering

In [379]:
import pandas as pd
import numpy as np

In [380]:
user_df = pd.read_csv('User_Listening_History.csv')
music_df = pd.read_csv('Music_Info.csv')

In [381]:
music_df.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [382]:
user_df.head()

,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


### 1. Wrangling & Cleaning

In [383]:
# change pc_limit to change the amount of playcounts
pc_limit = 20
user_df_sub = user_df[user_df['playcount'] <= pc_limit]

#### Genre encoding

In [384]:
# check for nulls
music_df.isnull().sum()

track_id                   0
name                       0
artist                     0
spotify_preview_url        0
spotify_id                 0
tags                    1127
genre                  28335
year                       0
duration_ms                0
danceability               0
energy                     0
key                        0
loudness                   0
mode                       0
speechiness                0
acousticness               0
instrumentalness           0
liveness                   0
valence                    0
tempo                      0
time_signature             0
dtype: int64

In [385]:
print(music_df['genre'].unique())

[nan 'RnB' 'Rock' 'Pop' 'Metal' 'Electronic' 'Jazz' 'Punk' 'Country'
 'Folk' 'Reggae' 'Rap' 'Blues' 'New Age' 'Latin' 'World']


In [386]:
music_df['tags'].value_counts()

tags
country                                                                             506
reggae                                                                              454
black_metal                                                                         442
rap, hip_hop                                                                        378
drum_and_bass                                                                       365
                                                                                   ... 
rock, alternative, alternative_rock, experimental, singer_songwriter, blues, 00s      1
indie, experimental, folk, acoustic, male_vocalists, guitar                           1
electronic, experimental, punk, industrial, punk_rock                                 1
rock, experimental, punk, punk_rock, 70s, grunge                                      1
rock, alternative_rock, japanese, cover                                               1
Name: count, Length: 20057,

In [387]:
# genre to lowercase
music_df['genre'] = music_df['genre'].str.lower()

# tags and genre null values empty string
music_df['tags'].fillna('', inplace=True)
music_df['genre'].fillna('', inplace=True)

In [388]:
def concatenate_genre_tags(row):
    genre = row['genre'].strip()
    tags = row['tags'].strip()

    if genre and genre not in tags:
        combined = f"{genre}, {tags}" if tags else genre
    else:
        combined = tags
    return combined


In [389]:
# make column tagsplus that is tags plus genre with comma and space
music_df['tagsplus'] = music_df.apply(concatenate_genre_tags, axis=1)

In [390]:
music_df['tagsplus']

0        rock, alternative, indie, alternative_rock, in...
1        rock, alternative, indie, pop, alternative_roc...
2        rnb, rock, alternative, alternative_rock, 90s,...
3        rock, alternative, indie, alternative_rock, in...
4        rnb, rock, alternative, indie, alternative_roc...
                               ...                        
50678                                                     
50679              rock, alternative_rock, japanese, cover
50680                      metal, metalcore, post_hardcore
50681                                                     
50682                           electronic, rock, japanese
Name: tagsplus, Length: 50683, dtype: object

In [391]:
# drop rows where tagsplus is empty string
music_df = music_df[music_df['tagsplus'] != '']

In [392]:
# for tagsplus count the amount of empty strings
music_df['tagsplus']

0        rock, alternative, indie, alternative_rock, in...
1        rock, alternative, indie, pop, alternative_roc...
2        rnb, rock, alternative, alternative_rock, 90s,...
3        rock, alternative, indie, alternative_rock, in...
4        rnb, rock, alternative, indie, alternative_roc...
                               ...                        
50675                          electronic, indie_pop, love
50677                          alternative_rock, punk_rock
50679              rock, alternative_rock, japanese, cover
50680                      metal, metalcore, post_hardcore
50682                           electronic, rock, japanese
Name: tagsplus, Length: 49988, dtype: object

In [393]:
len(music_df)

49988

### 2. Feature Engineering

In [394]:
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [395]:
# drop genre, tags and spotify fields
music_df.drop(columns=['genre', 'tags', 'spotify_preview_url', 'spotify_id'], inplace=True)

#### 2.1 Numerical scaling

In [396]:
# numeric features and transformer
numeric_features = ['year', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])


In [397]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

In [398]:
numeric_processed = preprocessor.fit_transform(music_df)

In [399]:
numeric_processed.shape

(49988, 14)

#### 2.2 Genre one-hot encoding

In [400]:
# tagsplus string to list
tagsplus_df = music_df['tagsplus'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

In [401]:
# use multilabelbinarizer to encode tagsplus
mlb = MultiLabelBinarizer()

genre_encoded = mlb.fit_transform(tagsplus_df)

genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=tagsplus_df.index)

In [402]:
genre_df.head()

,00s,60s,70s,80s,90s,acoustic,alternative,alternative_rock,ambient,american,...,soul,soundtrack,swedish,symphonic_metal,synthpop,techno,thrash_metal,trance,trip_hop,world
0,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [403]:
# re-merge list of numeric_features with genre_df dataframe
features_df = pd.concat([pd.DataFrame(numeric_processed, columns=numeric_features, index=music_df.index), genre_df], axis=1)


In [404]:
# add track_id, name and artist to features_df
features_df = pd.concat([music_df[['track_id', 'name', 'artist']], features_df], axis=1)


In [405]:
len(features_df)

49988

In [406]:
features_df.head()

,track_id,name,artist,year,duration_ms,danceability,energy,key,loudness,mode,...,soul,soundtrack,swedish,symphonic_metal,synthpop,techno,thrash_metal,trance,trip_hop,world
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.000849,-0.266861,-0.774918,0.921271,-1.208266,0.864513,0.763151,...,0,0,0,0,0,0,0,0,0,0
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.226113,0.071953,-0.472605,0.818013,-0.928036,0.861657,0.763151,...,0,0,0,0,0,0,0,0,0,0
2,TROUVHL128F426C441,Come as You Are,Nirvana,-1.463366,-0.297381,0.081637,0.555897,-0.367576,0.551888,-1.310357,...,0,0,0,0,0,0,0,0,0,0
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.000849,-0.128909,-1.200397,-0.087478,1.033573,-0.122133,0.763151,...,0,0,0,0,0,0,0,0,0,0
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.451377,-0.113891,0.120826,-1.016799,0.473114,-0.360281,0.763151,...,0,0,0,0,0,0,0,0,0,0


#### 2.3 User playcounts scaling

In [407]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler, specifying the range. scikit-surprise default is (1, 5)
scaler = MinMaxScaler(feature_range=(1, 5))

# Transform the playcount column
user_df['playcount_scale'] = scaler.fit_transform(user_df['playcount'].values.reshape(-1, 1))

In [408]:
# replace playcount w scaled
user_df['playcount'] = user_df['playcount_scale']
user_df.drop(columns=['playcount_scale'], inplace=True)

user_df.head()

,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1.0
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1.0
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1.0
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1.0
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1.0


#### 2.4 Dataset consistency

In [409]:
# remove tracks from user_df that are not in music_df
user_df = user_df[user_df['track_id'].isin(features_df['track_id'])]

# remove tracks from music_df that are not in user_df
features_df = features_df[features_df['track_id'].isin(user_df['track_id'])]

In [410]:
# sanity check
len(features_df['track_id'].unique()) == len(user_df['track_id'].unique())

True

In [411]:
# reset indices
user_df.reset_index(drop=True, inplace=True)
features_df.reset_index(drop=True, inplace=True)

#### 2.5 Encode track_id and user_id

In [412]:
# count unique track_id's in both dataframes
print('Number of unique track_id\'s in user_df: ', user_df['track_id'].nunique())
print('Number of unique track_id\'s in music_df: ', features_df['track_id'].nunique())

Number of unique track_id's in user_df:  30092
Number of unique track_id's in music_df:  30092


In [413]:
# encode track_id as numeric in both dataframes
track_id_to_numeric = {track_id: i for i, track_id in enumerate(features_df['track_id'])}

In [414]:
# double-check uniques
user_df['track_id_enc'] = user_df['track_id'].map(track_id_to_numeric)
features_df['track_id_enc'] = features_df['track_id'].map(track_id_to_numeric)

In [415]:
# sanity check: count unique track_id's in both dataframes... again
print('Number of unique track_id\'s in user_df: ', user_df['track_id_enc'].nunique())
print('Number of unique track_id\'s in music_df: ', features_df['track_id_enc'].nunique())


Number of unique track_id's in user_df:  30092
Number of unique track_id's in music_df:  30092


In [416]:
# test if encoding worked
# for track_id_enc = 10705, check if track_id in other dataframe has the same values
user_df[user_df['track_id_enc'] == 10705]['track_id'].unique() == features_df[features_df['track_id_enc'] == 10705]['track_id'].unique()

array([ True])

In [417]:
# unique user_id's in user_df
user_df['user_id'].nunique()

961452

In [418]:
# encode user_id in user_df dataframe
user_df['user_id'] = user_df['user_id'].astype('category')
user_df['user_id_enc'] = user_df['user_id'].cat.codes

In [419]:
# verify unique user_id's in user_df
user_df['user_id'].nunique()

961452

In [420]:
# replace original ids with encoded ids
user_df['track_id'] = user_df['track_id_enc']
user_df['user_id'] = user_df['user_id_enc']
features_df['track_id'] = features_df['track_id_enc']

In [421]:
# drop encoded columns
user_df.drop(columns=['track_id_enc', 'user_id_enc'], inplace=True)
features_df.drop(columns=['track_id_enc'], inplace=True)

### Wrapping up

In [422]:
user_df.head()

,track_id,user_id,playcount
0,16173,690941,1.0
1,1166,690941,1.0
2,22624,690941,1.0
3,2993,690941,1.0
4,1368,690941,1.0


In [423]:
features_df.head()

,track_id,name,artist,year,duration_ms,danceability,energy,key,loudness,mode,...,soul,soundtrack,swedish,symphonic_metal,synthpop,techno,thrash_metal,trance,trip_hop,world
0,0,Mr. Brightside,The Killers,0.000849,-0.266861,-0.774918,0.921271,-1.208266,0.864513,0.763151,...,0,0,0,0,0,0,0,0,0,0
1,1,Wonderwall,Oasis,0.226113,0.071953,-0.472605,0.818013,-0.928036,0.861657,0.763151,...,0,0,0,0,0,0,0,0,0,0
2,2,Take Me Out,Franz Ferdinand,0.000849,-0.128909,-1.200397,-0.087478,1.033573,-0.122133,0.763151,...,0,0,0,0,0,0,0,0,0,0
3,3,Karma Police,Radiohead,-0.900206,0.122692,-0.746926,-0.718940,0.473114,-0.183208,0.763151,...,0,0,0,0,0,0,0,0,0,0
4,4,Clocks,Coldplay,-0.224414,0.530362,0.467927,0.250095,-0.087346,0.237286,-1.310357,...,0,0,0,0,0,0,0,0,0,0


In [424]:
# dataframe of track_id, name and artist
titles = features_df[['track_id', 'name', 'artist']]

In [425]:
# test numbers
len(features_df) == user_df['track_id'].nunique() == len(titles)

True

In [426]:
import pickle
pickle.dump(features_df, open('features.pkl', 'wb'))
pickle.dump(user_df, open('playcounts.pkl', 'wb'))
pickle.dump(titles, open('titles.pkl', 'wb'))